In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
import sys
import json
import cv2
import albumentations as A



module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from building_footprint_segmentation.segmentation import init_segmentation, read_trainer_config
from building_footprint_segmentation.helpers.callbacks import CallbackList, load_callback, BinaryTestCallback
from building_footprint_segmentation.trainer import Trainer
from building_footprint_segmentation.utils.py_network import convert_tensor_to_numpy

/home/leadawon5/dawon/bsvenv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initalize type of segmentation to run for building footprint

In [2]:
segmentation = init_segmentation("binary")

### Defining augmenters to use

In [3]:
augmenters = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2)
])

### Load essential segmentation objects for training

In [4]:
!pwd

/home/leadawon5/dawon/vision/dacon/building-footprint-segmentation/examples


In [5]:
model = segmentation.load_model(name="ReFineNet")
criterion = segmentation.load_criterion(name="BinaryCrossEntropy")
loader = segmentation.load_loader( #seg/base_loader.py --> def get_data_loader
    root_folder="/home/leadawon5/dawon/vision/dacon/building-footprint-segmentation/data/schange/",
    image_normalization="divide_by_255",
    label_normalization="binary_label",
    augmenters=augmenters,
    batch_size=2,
)
metrics = segmentation.load_metrics(
    data_metrics=["accuracy", "precision", "f1", "recall", "iou"]
)

optimizer = segmentation.load_optimizer(model, name="Adam")

callbacks = CallbackList([BinaryTestCallback("../callback")])

# Ouptut from all the callbacks caller will be stored at the path specified in log_dir
for caller in  ["TrainChkCallback", "TimeCallback", "TensorBoardCallback", "TrainStateCallback", "BinaryTestCallback"]:
    callbacks.append(load_callback("../callback", caller))

/home/leadawon5/dawon/bsvenv/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/home/leadawon5/dawon/bsvenv/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Initalize the Trainer object

In [6]:
trainer = Trainer(
    model=model,
    criterion=criterion,
    loader=loader,
    metrics=metrics,
    callbacks=callbacks,
    optimizer=optimizer,
    scheduler=None,
)

### Start training

In [7]:
trainer.train(start_epoch=0, end_epoch=50)

Epoch 0, lr 0.001:   0%|          | 0/138 [00:00<?, ?it/s]

Epoch 0, lr 0.001: 100%|██████████| 138/138 [00:08<00:00, 15.45it/s, loss=593.87267]


>>Py-OneLiner : Loss : train: 593.8726722619672, validation: 1.7637795805931091
>>Py-OneLiner : Train Metric : accuracy : 0.72012, precision : 0.06410, f1 : 0.06289, recall : 0.21723, iou : 0.03506
>>Py-OneLiner : Valid Metric : accuracy : 0.76502, precision : 0.09632, f1 : 0.12319, recall : 0.17108, iou : 0.06582

Epoch 1, lr 0.001: 100%|██████████| 138/138 [00:05<00:00, 24.25it/s, loss=0.71696]

>>Py-OneLiner : Validation : 1/2 ETA -- 0:Days 0:Hours 0:Minutes 0:Seconds

...

>>Py-OneLiner : Validation : 1/2 ETA -- 0:Days 0:Hours 0:Minutes 0:Seconds

>>Py-OneLiner : Loss : train: 0.3589210592318272, validation: 0.31353069841861725
>>Py-OneLiner : Train Metric : accuracy : 0.88358, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Valid Metric : accuracy : 0.90426, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000

Epoch 49, lr 0.001: 100%|██████████| 138/138 [00:04<00:00, 28.69it/s, loss=0.35984]

>>Py-OneLiner : Validation : 1/2 ETA -- 0:Days 0:Hours 0:Minutes 0:Seconds

>>Py-OneLiner : Loss : train: 0.3598354657491048, validation: 0.3095617890357971
>>Py-OneLiner : Train Metric : accuracy : 0.88323, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Valid Metric : accuracy : 0.90454, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Training Complete
>>Py-OneLiner : Run Time : 0:Days 0:Hours 14:Minutes 47:Seconds